In [100]:
import pandas as pd
from pulp import *

In [101]:
df = pd.read_excel("First_Draft.xlsx")
df

,Team,Player ID,First Name,Second Name,Web Name,Position,Start Price,Current Price,Selected By,Transfers In,Transfers Out,Total Points,Bonus,Minutes,Goals Scored,Assists,Clean Sheets,Status,Form
0,Arsenal,1,Cédric,Alves Soares,Cédric,Defender,-0.1,4.4,0.2,2095,10477,0,0,0,0,0,0,a,0.0
1,Arsenal,3,Granit,Xhaka,Xhaka,Midfielder,0.0,5.0,2.0,112293,27712,14,2,180,1,1,1,a,7.0
2,Arsenal,4,Mohamed,Elneny,Elneny,Midfielder,-0.1,4.4,1.3,44873,44144,0,0,0,0,0,0,a,0.0
3,Arsenal,5,Rob,Holding,Holding,Defender,-0.1,4.4,0.2,1197,5742,0,0,0,0,0,0,a,0.0
4,Arsenal,6,Thomas,Partey,Partey,Midfielder,0.0,5.0,0.7,19392,19522,5,0,180,0,0,1,a,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,Wolves,566,Joseph,Hodge,Hodge,Midfielder,0.0,4.5,0.0,1805,851,0,0,0,0,0,0,a,0.0
585,Wolves,567,Connor,Ronan,Ronan,Midfielder,0.0,4.5,0.1,4264,1913,0,0,0,0,0,0,a,0.0
586,Wolves,568,Chem,Campbell,Campbell,Midfielder,0.0,4.5,0.3,15737,6455,1,0,5,0,0,0,a,0.5
587,Wolves,579,Gonçalo Manuel,Ganchinho Guedes,Guedes,Midfielder,0.0,6.0,0.1,10123,3130,1,0,32,0,0,0,a,1.0


In [102]:
df = df[['Web Name', 'Team', 'Position', 'Total Points', 'Current Price', 'Minutes']]
df = df.rename(columns={'Web Name': 'Name', 'Total Points': 'Points', 'Current Price': 'Price'})

In [103]:

POS = df.Position.unique()
CLUBS = df.Team.unique()
BUDGET = 100
pos_available = {
    'Defender': 5,
    'Forward': 3,
    'Midfielder': 5,
    'Goalkeeper': 2,
}

# Initialize Variables
names = [df.Name[i] for i in df.index]
teams =  [df.Team[i] for i in df.index]
positions =  [df.Position[i] for i in df.index]
prices =  [df.Price[i] for i in df.index]
points =  [df.Points[i] for i in df.index]
minutes = [df.Minutes[i] for i in df.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in df.index]

In [104]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [105]:
# Build the constraints
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(df))) # Objective

prob += lpSum(players[i] * df.Price[df.index[i]] for i in range(len(df))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(df)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(df)) if teams[i] == club) <= 3 # Club Limit

In [106]:
# Solve the problem
prob.solve()

1

In [107]:
#print results as a dataframe
df_result = pd.DataFrame(columns=['Name', 'Team', 'Position', 'Points', 'Price'])
for v in prob.variables():
    if v.varValue != 0:
        name = df.Name[int(v.name.split("_")[1])]
        club = df.Team[int(v.name.split("_")[1])]
        position = df.Position[int(v.name.split("_")[1])]
        point = df.Points[int(v.name.split("_")[1])]
        price = df.Price[int(v.name.split("_")[1])]
        minutes = df.Minutes[int(v.name.split("_")[1])]
        df_result = df_result.append({'Name': name, 'Team': club, 'Position': position, 'Points': point, 'Price': price, 'Minutes': minutes}, ignore_index=True)

In [108]:

#Points per minute
df_result['Points per million'] = df_result['Points'] / df_result['Price'].round(2)

df_result.sort_values(by=['Points per million'], ascending=False, inplace=True)
df_result

,Name,Team,Position,Points,Price,Minutes,Points per million
10,Schär,Newcastle Utd,Defender,20,4.6,180.0,4.347826
11,Henderson,Nottingham Forest,Goalkeeper,17,4.5,180.0,3.777778
5,Rodrigo,Leeds United,Midfielder,22,6.1,154.0,3.606557
12,Sá,Wolves,Goalkeeper,17,5.0,180.0,3.4
1,Groß,Brighton,Midfielder,18,5.6,180.0,3.214286
0,Mee,Brentford,Defender,14,4.5,148.0,3.111111
14,Jensen,Brentford,Midfielder,15,5.0,131.0,3.0
9,Trippier,Newcastle Utd,Defender,14,5.0,180.0,2.8
4,Jesus,Arsenal,Forward,21,8.1,165.0,2.592593
3,Martinelli,Arsenal,Midfielder,16,6.3,180.0,2.539683
